In [1]:
import sys

sys.path.append("..")

import nivapy3 as nivapy
import pandas as pd
import teotil2 as teo
import useful_rid_code as rid

# Generate overall summary table - 2021

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway and for the four OSPAR regions. This table no longer appears in recent reports, but I find it to be a useful summary document. This notebook reads the output from the data processing so far and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

In [2]:
# Year of interest
year = 2021

## 1. Summarise monitoring data

### 1.1. Identify network nodes with monitoring data

In [3]:
# Read station data
in_xlsx = r"../../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=["ospar_region", "nve_vassdrag_nr"])

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network
ann_input_csv = (
    f"../../../../teotil2/data/norway_annual_input_data/input_data_{year}.csv"
)
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

### 1.2. Read monitored fluxes

In [4]:
# Read data
in_csv = f"../../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
# mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "new_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("new_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "new_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"]

# Rename cols to match template
mon_df["Flow rate_1000m3/day"] = mon_df["mean_q_1000m3/day"]
del mon_df["mean_q_1000m3/day"]

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr    Cu     Hg  NH4-N  \
ospar_region        new_rid_group                                             
LOFOTEN-BARENTS SEA rid_135        0.0   1.0  0.0   1.0   6.0    5.0   74.0   
                    rid_20         0.0   1.0  0.0   5.0  13.0   15.0   76.0   
NORTH SEA           rid_135        0.0   2.0  0.0   2.0  13.0   17.0  199.0   
                    rid_20         0.0   0.0  0.0   0.0   1.0    5.0   29.0   
NORWEGIAN SEA2      rid_135        0.0   3.0  0.0   8.0  25.0   24.0  267.0   
                    rid_20         0.0   1.0  0.0   2.0  11.0    9.0   31.0   
SKAGERAK            rid_135        0.0   2.0  0.0   1.0   7.0   10.0  159.0   
                    rid_20         0.0  11.0  1.0  26.0  67.0  119.0  584.0   
NORWAY              rid_135        0.0   8.0  1.0  12.0  50.0   56.0  698.0   
                    rid_20         0.0  13.0  1.0  33.0  93.0  149.0  721.0   

                                     NO3-N    Ni  PO4-P    Pb       SPM  \
ospar_region        new_rid_group                                         
LOFOTEN-BARENTS SEA rid_135          245.0   5.0   13.0   0.0   12535.0   
                    rid_20           553.0  20.0   97.0   1.0  409888.0   
NORTH SEA           rid_135         3940.0   7.0   49.0   4.0   44253.0   
                    rid_20           741.0   1.0   11.0   1.0    6098.0   
NORWEGIAN SEA2      rid_135         2314.0  18.0   80.0   2.0  103423.0   
                    rid_20           856.0   4.0   37.0   0.0   49528.0   
SKAGERAK            rid_135         2356.0   4.0   28.0   3.0   23670.0   
                    rid_20         17205.0  47.0  397.0  17.0  384387.0   
NORWAY              rid_135         8855.0  33.0  170.0  10.0  183881.0   
                    rid_20         19355.0  72.0  543.0  19.0  849900.0   

                                       SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135         20015.0   23180.0   1125.0   23.0    6.0   
                    rid_20          93267.0   52122.0   3237.0  185.0   24.0   
NORTH SEA           rid_135         36883.0   42635.0   6911.0  137.0   47.0   
                    rid_20           5104.0    7058.0   1132.0   31.0    8.0   
NORWEGIAN SEA2      rid_135         46712.0   74290.0   6822.0  183.0   57.0   
                    rid_20          23413.0   21047.0   1801.0   66.0   26.0   
SKAGERAK            rid_135         20845.0   47890.0   4612.0  106.0   44.0   
                    rid_20         182293.0  204073.0  25543.0  700.0  218.0   
NORWAY              rid_135        124455.0  187995.0  19470.0  449.0  154.0   
                    rid_20         304077.0  284301.0  31713.0  982.0  275.0   

                                   Flow rate  
ospar_region        new_rid_group             
LOFOTEN-BARENTS SEA rid_135          28984.0  
                    rid_20           52550.0  
NORTH SEA           rid_135          82505.0  
                    rid_20           11299.0  
NORWEGIAN SEA2      rid_135          96308.0  
                    rid_20           29605.0  
SKAGERAK            rid_135          26554.0  
                    rid_20          144007.0  
NORWAY              rid_135         234351.0  
                    rid_20          237461.0

## 2. Data for unmonitored areas

In [5]:
# Read data
in_csv = f"../../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,327178.0,13013.0,1100.0,2360.0,227.0,70816.0,12210.0,31138.0,679.0,660.0,...,12821.0,0.0,0.0,0.0,6.0,20.0,2.0,1.0,5.0,1308.0
LOFOTEN-BARENTS SEA,105079.0,1442.0,164.0,125.0,11.0,18948.0,3260.0,4898.0,99.0,99.0,...,2324.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,349.0
NORTH SEA,93125.0,3444.0,466.0,441.0,83.0,20875.0,3594.0,11615.0,195.0,280.0,...,4316.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,384.0
NORWEGIAN SEA2,120626.0,2804.0,383.0,899.0,95.0,30955.0,5349.0,11706.0,286.0,230.0,...,4203.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,4.0,574.0
SKAGERAK,8348.0,5322.0,86.0,895.0,38.0,37.0,6.0,2919.0,100.0,52.0,...,1978.0,0.0,0.0,0.0,4.0,14.0,1.0,0.0,1.0,1.0


## 3. Create table

The cell below uses the 3-fold division of sites (`'rid_11'` etc.) from the old progreamme structure. To use this, copy the old Word template and make sure that the aggregation in Section 1.2 is done by `'old_rid_group'`.

In [6]:
## Create table 3 (old programme structure)
# in_docx = f'../../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx'
#
# rid.write_word_overall_table(mon_df, umon_df, in_docx)

The cell below uses the 2-fold division of sites (`'rid_20'` and `'rid_135'`) for the 2017-20 progreamme structure. To use this, copy the new Word template and make sure that the aggregation in Section 1.2 is done by `'new_rid_group'`.

In [8]:
# Create table 3
in_docx = (
    f"../../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx"
)
rid.write_word_overall_table_2017_20(mon_df, umon_df, in_docx)

Finished.
